In [64]:
from pandas.core.algorithms import SelectN
import pyomo.environ as pyo, numpy as np, pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [65]:
# entradas
barras = pd.read_excel('sist_eletrico.xlsx', sheet_name='barras')
geracao = pd.read_excel('sist_eletrico.xlsx', sheet_name='geracao')
carga = pd.read_excel('sist_eletrico.xlsx', sheet_name='carga')
linha = pd.read_excel('sist_eletrico.xlsx', sheet_name='linha')
Nb = len(barras)
Ng = len(geracao)
Nd = len(carga)
Nl = len(linha)

In [66]:
# modelo
model = pyo.ConcreteModel()
model.Pg = pyo.Var(range(Ng))
model.Pl = pyo.Var(range(Nl))
model.teta = pyo.Var(range(Nb), bounds=(-np.pi, np.pi))
Pg = model.Pg
Pl = model.Pl
teta = model.teta

In [67]:
# objetivo
model.obj = pyo.Objective(expr=sum([Pg[g]*geracao.custo[g] for g in geracao.index]), sense=minimize)

In [68]:
# balanço de potência
model.balanco = pyo.ConstraintList()
for n in barras.index:
    sum_Pg = sum([Pg[g] for g in geracao.index[geracao.barra==n]])
    sum_Pls = sum([Pl[l] for l in linha.index[linha.barra_de==n]])
    sum_Plr = sum([Pl[l] for l in linha.index[linha.barra_para==n]])
    sum_Pd = sum([carga.carga[d] for d in carga.index[carga.barra==n]])
    model.balanco.add(expr=sum_Pg - sum_Pls + sum_Plr == sum_Pd)

In [69]:
# fluxo de potência
model.fluxo = pyo.ConstraintList()
for l in linha.index:
    Bl = linha.Bl[l]
    n_send = linha.barra_de[l]
    n_rec = linha.barra_para[l]
    delta_teta = teta[n_send] - teta[n_rec]
    model.fluxo.add(expr=Pl[l] == Bl*delta_teta)

In [70]:
# Restrições limites gerador
model.limger = pyo.ConstraintList()
for g in geracao.index:
    model.limger.add(inequality(0,Pg[g], geracao.pgmax[g]))

In [71]:
# limites fluxo de potência
model.limflux = pyo.ConstraintList()
for l in linha.index:
    model.limflux.add(inequality(-linha.plmax[l], Pl[l], linha.plmax[l]))

In [72]:
# referência
model.ref = pyo.Constraint(expr= teta[0]==0)

In [73]:
# solve
opt = SolverFactory('gurobi')
opt.solve(model)

{'Problem': [{'Name': 'x9', 'Lower bound': 6.5, 'Upper bound': 6.5, 'Number of objectives': 1, 'Number of constraints': 18, 'Number of variables': 9, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 9, 'Number of nonzeros': 29, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.0', 'Error rc': 0, 'Time': 0.15060806274414062}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [74]:
# result
model.pprint()

7 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {0, 1}
    Pl_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    balanco_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    fluxo_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    limflux_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    limger_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    teta_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : S